# Model Context Protocol (MCP) Integration Guide

## ✅ Overview

This comprehensive guide explores the **Model Context Protocol (MCP)** integration in the Strands SDK. MCP is a revolutionary protocol that enables AI agents to seamlessly connect with external tools, resources, and data sources.

### What You'll Learn:
- Core MCP concepts and architecture
- Advanced timeout and connection management
- Tool discovery and execution patterns
- Prompt management capabilities (NEW)
- Async/sync execution modes
- Transport options and best practices
- Real-world integration examples

### Prerequisites:
- Python 3.10+
- Strands SDK installed
- Basic understanding of async/await patterns

## 1. Setup and Installation

First, let's ensure we have all required packages:

In [ ]:
# Install required packages if not already installed
import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import strands
    from strands.tools.mcp import MCPClient
    print("✅ Strands SDK already installed")
except ImportError:
    print("Installing strands-agents...")
    install_package("strands-agents")

try:
    import mcp
    print("✅ MCP SDK already installed")
except ImportError:
    print("Installing mcp...")
    install_package("mcp")

In [ ]:
# Core imports
import asyncio
import json
import threading
import time
from datetime import timedelta
from typing import Dict, Any, List

# Strands imports
from strands import Agent
from strands.tools.mcp import MCPClient, MCPTransport
from strands.tools.mcp.mcp_types import MCPToolResult

# MCP SDK imports
from mcp import StdioServerParameters, stdio_client
from mcp.client.sse import sse_client
from mcp.client.streamable_http import streamablehttp_client
from mcp.server import FastMCP
from mcp.types import ImageContent as MCPImageContent

print("✅ All imports successful!")

## 2. Core MCP Architecture

### MCPClient: The Heart of MCP Integration

The `MCPClient` is the cornerstone of MCP integration in Strands. It provides:

- **Background Thread Management**: Runs MCP communication in a separate thread
- **Context Manager Pattern**: Ensures proper resource cleanup
- **Transport Abstraction**: Supports multiple communication protocols
- **Timeout Control**: Configurable startup and execution timeouts

### Architecture Diagram

```
┌─────────────────┐    ┌──────────────────┐    ┌─────────────────┐
│   Strands       │    │    MCPClient     │    │   MCP Server    │
│   Agent         │◄──►│                  │◄──►│                 │
│                 │    │ Background Thread│    │  Tools/Prompts  │
└─────────────────┘    └──────────────────┘    └─────────────────┘
        │                       │                       │
        │                   Transport                   │
        └───────────────────────┼───────────────────────┘
                           stdio/SSE/HTTP
```

## 3. Creating a Simple MCP Server for Testing

Let's create a demonstration MCP server to showcase the features:

In [ ]:
def create_demo_mcp_server(port: int = 8000):
    """
    Creates a demonstration MCP server with various tools and prompts.
    This server showcases different MCP capabilities.
    """
    mcp = FastMCP("Demo MCP Server", port=port)
    
    # Basic calculation tool
    @mcp.tool(description="Performs mathematical calculations")
    def calculator(operation: str, x: float, y: float) -> float:
        """Calculate mathematical operations"""
        operations = {
            'add': lambda a, b: a + b,
            'subtract': lambda a, b: a - b,
            'multiply': lambda a, b: a * b,
            'divide': lambda a, b: a / b if b != 0 else float('inf')
        }
        return operations.get(operation, lambda a, b: 0)(x, y)
    
    # Tool with structured output (NEW FEATURE)
    @mcp.tool(description="Analyzes text and returns structured data", structured_output=True)
    def text_analyzer(text: str) -> Dict[str, Any]:
        """Analyze text and return structured metrics"""
        words = text.split()
        return {
            "word_count": len(words),
            "character_count": len(text),
            "character_count_no_spaces": len(text.replace(" ", "")),
            "average_word_length": sum(len(word) for word in words) / len(words) if words else 0,
            "longest_word": max(words, key=len) if words else "",
            "sentiment": "positive" if "good" in text.lower() or "great" in text.lower() else "neutral"
        }
    
    # Prompt templates (NEW FEATURE)
    @mcp.prompt(description="Generates a greeting prompt")
    def greeting_prompt(name: str = "World", style: str = "formal") -> str:
        """Generate a personalized greeting"""
        styles = {
            "formal": f"Good day, {name}. I trust you are well.",
            "casual": f"Hey {name}! How's it going?",
            "enthusiastic": f"Hello there, amazing {name}! Ready for an awesome day?"
        }
        return styles.get(style, styles["formal"])
    
    @mcp.prompt(description="Creates a code review prompt")
    def code_review_prompt(language: str = "python", focus: str = "general") -> str:
        """Generate a code review prompt"""
        focuses = {
            "performance": f"Please review this {language} code focusing on performance optimizations and efficiency.",
            "security": f"Please review this {language} code focusing on security vulnerabilities and best practices.",
            "style": f"Please review this {language} code focusing on coding style and conventions.",
            "general": f"Please provide a comprehensive review of this {language} code."
        }
        return focuses.get(focus, focuses["general"])
    
    return mcp

print("✅ Demo MCP server functions created")

## 4. Basic MCPClient Usage

### Creating and Using MCPClient

The MCPClient uses a context manager pattern for proper resource management:

In [ ]:
# Let's start with a simple stdio-based MCP server
# We'll use the echo server from the Strands test suite

def create_echo_server_client():
    """
    Creates an MCPClient connected to a simple echo server.
    This demonstrates the basic connection pattern.
    """
    return MCPClient(
        lambda: stdio_client(
            StdioServerParameters(
                command="python", 
                args=["-c", """
                import sys
                from mcp.server import FastMCP

                mcp = FastMCP("Echo Server")

                @mcp.tool(description="Echoes back the input")
                def echo(message: str) -> str:
                    return f"Echo: {message}"

                @mcp.tool(description="Echoes back with structured content", structured_output=True)
                def echo_structured(message: str) -> dict:
                    return {"echoed": message, "timestamp": "2024-01-01T00:00:00Z"}

                mcp.run(transport="stdio")
                """]
            )
        )
    )

# Create the client
echo_client = create_echo_server_client()
print("✅ Echo server MCP client created")

In [ ]:
# Using the context manager pattern
with echo_client:
    # List available tools
    tools = echo_client.list_tools_sync()
    print(f"✅ Found {len(tools)} tools:")
    
    for tool in tools:
        print(f"  - {tool.tool_name}: {tool.tool_spec['description']}")
    
    # Test a simple tool call
    result = echo_client.call_tool_sync(
        tool_use_id="test-001",
        name="echo",
        arguments={"message": "Hello, MCP!"}
    )
    
    print(f"\n✅ Tool call result:")
    print(f"  Status: {result['status']}")
    print(f"  Content: {result['content']}")

## 5. Timeout Configuration (NEW FEATURE)

The MCPClient now supports comprehensive timeout configuration:

In [ ]:
# Timeout configuration examples

# 1. Startup timeout configuration
slow_startup_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="python", 
            args=["-c", "import time; time.sleep(2); from mcp.server import FastMCP; mcp = FastMCP('Slow Server'); mcp.run(transport='stdio')"]
        )
    ),
    startup_timeout=60  # Wait up to 60 seconds for server startup
)

print("✅ Client with custom startup timeout created")

# 2. Tool execution timeout demonstration
with echo_client:
    try:
        # Call tool with execution timeout
        result = echo_client.call_tool_sync(
            tool_use_id="timeout-test",
            name="echo",
            arguments={"message": "Testing timeout"},
            read_timeout_seconds=timedelta(seconds=5)  # 5-second timeout
        )
        print("✅ Tool call with timeout completed successfully")
        print(f"  Result: {result['content'][0]['text']}")
    except Exception as e:
        print(f"❌ Tool call failed: {e}")

## 6. Async Tool Execution (NEW FEATURE)

MCPClient now supports both synchronous and asynchronous tool execution:

In [ ]:
# Async tool execution example
async def demonstrate_async_calls():
    """Demonstrate async MCP tool calls"""
    
    # Use the echo client in async context
    with echo_client:
        print("🔄 Starting async tool calls...")
        
        # Execute multiple tool calls concurrently
        tasks = []
        for i in range(3):
            task = echo_client.call_tool_async(
                tool_use_id=f"async-{i}",
                name="echo",
                arguments={"message": f"Async message {i+1}"}
            )
            tasks.append(task)
        
        # Wait for all calls to complete
        results = await asyncio.gather(*tasks)
        
        print("✅ All async calls completed:")
        for i, result in enumerate(results):
            print(f"  Task {i+1}: {result['content'][0]['text']}")

# Run the async demonstration
await demonstrate_async_calls()

## 7. Prompt Management (NEW FEATURE)

MCPClient now supports prompt discovery and retrieval:

In [ ]:
# Create a server with prompts for demonstration
def create_prompt_server_client():
    return MCPClient(
        lambda: stdio_client(
            StdioServerParameters(
                command="python", 
                args=["-c", """
from mcp.server import FastMCP

mcp = FastMCP("Prompt Server")

@mcp.prompt(description="Generates a greeting prompt")
def greeting_prompt(name: str = "World", style: str = "formal") -> str:
    styles = {
        "formal": f"Good day, {name}. I trust you are well.",
        "casual": f"Hey {name}! How's it going?",
        "enthusiastic": f"Hello there, {name}! Ready for an awesome day?"
    }
    return styles.get(style, styles["formal"])

@mcp.prompt(description="Creates a writing prompt")
def writing_prompt(genre: str = "fiction", length: str = "short") -> str:
    return f"Write a {length} {genre} story about an unexpected discovery."

mcp.run(transport="stdio")
"""]
            )
        )
    )

prompt_client = create_prompt_server_client()
print("✅ Prompt server client created")

In [ ]:
# Demonstrate prompt management
with prompt_client:
    # List available prompts
    prompts_result = prompt_client.list_prompts_sync()
    print(f"✅ Found {len(prompts_result.prompts)} prompts:")
    
    for prompt in prompts_result.prompts:
        print(f"  - {prompt.name}: {prompt.description}")
    
    # Get a specific prompt with parameters
    if prompts_result.prompts:
        prompt_name = prompts_result.prompts[0].name
        prompt_result = prompt_client.get_prompt_sync(
            prompt_name, 
            {"name": "Alice", "style": "enthusiastic"}
        )
        
        print(f"\n✅ Prompt '{prompt_name}' result:")
        for message in prompt_result.messages:
            print(f"  {message.content.text}")
    
    # Demonstrate pagination (NEW FEATURE)
    paginated_prompts = prompt_client.list_prompts_sync(pagination_token=None)
    print(f"\n✅ Pagination support: {len(paginated_prompts.prompts)} prompts loaded")
    if prompts_result.nextCursor:
        print(f"  Next cursor available: {prompts_result.nextCursor[:20]}...")

## 8. Structured Content Support

MCP tools can return structured content alongside regular text responses:

In [ ]:
# Test structured content with the echo server
with echo_client:
    # Call a tool that returns structured content
    structured_result = echo_client.call_tool_sync(
        tool_use_id="structured-test",
        name="echo_structured",
        arguments={"message": "Testing structured content"}
    )
    
    print("✅ Structured content result:")
    print(f"  Status: {structured_result['status']}")
    print(f"  Regular content: {structured_result['content']}")
    
    # Check for structured content (NEW FEATURE)
    if 'structuredContent' in structured_result:
        print(f"  ✅ Structured content found:")
        print(f"    {json.dumps(structured_result['structuredContent'], indent=2)}")
    else:
        print("  ❌ No structured content available")

## 9. Transport Options

MCPClient supports multiple transport mechanisms:

In [ ]:
# 1. stdio Transport (most common)
stdio_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx",
            args=["mcp-server-git"]  # Example: Git MCP server
        )
    )
)

print("✅ stdio transport client created")

# 2. SSE (Server-Sent Events) Transport
# Note: Requires a running SSE MCP server
def create_sse_client(url: str):
    return MCPClient(lambda: sse_client(url))

print("✅ SSE transport function created")

# 3. Streamable HTTP Transport
# Note: Requires a running HTTP MCP server
def create_http_client(url: str):
    return MCPClient(lambda: streamablehttp_client(url=url))

print("✅ HTTP transport function created")

# Transport comparison
transport_info = {
    "stdio": {
        "description": "Standard input/output communication",
        "use_case": "Local tools and simple integrations",
        "pros": ["Simple setup", "No network required", "Direct process communication"],
        "cons": ["Single process only", "No web integration"]
    },
    "SSE": {
        "description": "Server-Sent Events over HTTP",
        "use_case": "Web-based tools and remote servers",
        "pros": ["Web-compatible", "Real-time updates", "Firewall-friendly"],
        "cons": ["Requires HTTP server", "More complex setup"]
    },
    "HTTP": {
        "description": "Streamable HTTP transport",
        "use_case": "Enterprise integrations and cloud services",
        "pros": ["Full HTTP features", "Load balancing support", "Enterprise ready"],
        "cons": ["Most complex setup", "Requires HTTP infrastructure"]
    }
}

print("\n📋 Transport Options Summary:")
for transport, info in transport_info.items():
    print(f"\n{transport.upper()} Transport:")
    print(f"  Description: {info['description']}")
    print(f"  Use case: {info['use_case']}")
    print(f"  Pros: {', '.join(info['pros'])}")
    print(f"  Cons: {', '.join(info['cons'])}")

## 10. Integration with Strands Agents

The real power of MCP comes from integrating external tools with Strands agents:

In [ ]:
# Create an agent with MCP tools
with echo_client:
    # Get all available tools from the MCP server
    mcp_tools = echo_client.list_tools_sync()
    
    # Create an agent with these tools
    agent = Agent(tools=mcp_tools)
    
    print(f"✅ Agent created with {len(mcp_tools)} MCP tools")
    
    # Use the agent to interact with MCP tools
    response = agent("Please echo back the message: 'MCP integration is working!'")
    
    print("\n🤖 Agent Response:")
    print(response.message['content'][0]['text'])
    
    # Check if tools were actually used
    tool_uses = []
    for message in agent.messages:
        for content in message['content']:
            if 'toolUse' in content:
                tool_uses.append(content['toolUse']['name'])
    
    if tool_uses:
        print(f"\n✅ Tools used: {', '.join(set(tool_uses))}")
    else:
        print("\n❌ No tools were used")

## 11. Error Handling and Resilience

Proper error handling is crucial for robust MCP integrations:

In [ ]:
from strands.types.exceptions import MCPClientInitializationError

# Error handling examples

# 1. Handling initialization errors
try:
    bad_client = MCPClient(
        lambda: stdio_client(
            StdioServerParameters(
                command="nonexistent-command",
                args=[]
            )
        ),
        startup_timeout=5  # Short timeout for demo
    )
    
    with bad_client:
        tools = bad_client.list_tools_sync()
        
except MCPClientInitializationError as e:
    print(f"✅ Caught initialization error (expected): {e}")
except Exception as e:
    print(f"✅ Caught other error (expected): {type(e).__name__}: {e}")

# 2. Handling tool execution errors
with echo_client:
    try:
        # Call a non-existent tool
        result = echo_client.call_tool_sync(
            tool_use_id="error-test",
            name="nonexistent_tool",
            arguments={}
        )
        print(f"Tool call result status: {result['status']}")
        if result['status'] == 'error':
            print(f"✅ Error handled gracefully: {result['content'][0]['text']}")
    except Exception as e:
        print(f"✅ Exception caught: {e}")

# 3. Best practices for error handling
def robust_mcp_call(client: MCPClient, tool_name: str, arguments: dict, max_retries: int = 3):
    """Example of robust MCP tool calling with retries"""
    for attempt in range(max_retries):
        try:
            result = client.call_tool_sync(
                tool_use_id=f"robust-{attempt}",
                name=tool_name,
                arguments=arguments,
                read_timeout_seconds=timedelta(seconds=10)
            )
            
            if result['status'] == 'success':
                return result
            else:
                print(f"Attempt {attempt + 1} failed: {result['content'][0]['text']}")
                
        except Exception as e:
            print(f"Attempt {attempt + 1} exception: {e}")
            if attempt == max_retries - 1:
                raise
            time.sleep(1)  # Brief delay before retry
    
    return None

print("✅ Robust error handling examples completed")

## 12. Client Reuse Pattern

MCPClient supports reuse across multiple sessions:

In [ ]:
# Demonstrate client reuse
reusable_client = create_echo_server_client()

# First session
with reusable_client:
    tools1 = reusable_client.list_tools_sync()
    result1 = reusable_client.call_tool_sync(
        tool_use_id="reuse-1",
        name="echo",
        arguments={"message": "First session"}
    )
    print(f"✅ First session: {result1['content'][0]['text']}")

# Second session (reusing the same client)
with reusable_client:
    tools2 = reusable_client.list_tools_sync()
    result2 = reusable_client.call_tool_sync(
        tool_use_id="reuse-2",
        name="echo",
        arguments={"message": "Second session"}
    )
    print(f"✅ Second session: {result2['content'][0]['text']}")

print(f"✅ Client reuse successful - both sessions found {len(tools1)} tools")

## 13. Real-World Example: AWS Documentation Server

Let's demonstrate integration with a real MCP server:

In [ ]:
# AWS Documentation MCP Server example
# Note: This requires the AWS documentation MCP server to be available

def create_aws_docs_client():
    """Create a client for AWS documentation MCP server"""
    return MCPClient(
        lambda: stdio_client(
            StdioServerParameters(
                command="uvx",
                args=["awslabs.aws-documentation-mcp-server@latest"]
            )
        ),
        startup_timeout=60  # AWS server may take longer to start
    )

# Uncomment the following to test with AWS documentation server
# (requires internet connection and uvx)
"""
try:
    aws_client = create_aws_docs_client()
    
    with aws_client:
        aws_tools = aws_client.list_tools_sync()
        print(f"✅ AWS Documentation server has {len(aws_tools)} tools available")
        
        # Create an agent with AWS documentation tools
        aws_agent = Agent(tools=aws_tools)
        
        # Query about AWS services
        response = aws_agent("Tell me about Amazon Bedrock and how to use it with Python")
        print("\n🔍 AWS Documentation Query Result:")
        print(response.message['content'][0]['text'][:500] + "...")
        
except Exception as e:
    print(f"AWS documentation server not available: {e}")
"""

print("✅ AWS documentation example prepared (uncomment to run)")

## 14. Performance Considerations and Best Practices

### Key Performance Tips:

1. **Connection Reuse**: Reuse MCPClient instances when possible
2. **Async Operations**: Use async methods for concurrent tool calls
3. **Appropriate Timeouts**: Set reasonable timeouts based on expected tool execution time
4. **Error Handling**: Implement robust error handling and retry logic
5. **Resource Cleanup**: Always use context managers for proper cleanup

In [ ]:
# Performance benchmarking example
import time

def benchmark_mcp_calls(client: MCPClient, num_calls: int = 5):
    """Benchmark MCP tool calls"""
    
    with client:
        # Synchronous calls
        start_time = time.time()
        for i in range(num_calls):
            result = client.call_tool_sync(
                tool_use_id=f"bench-sync-{i}",
                name="echo",
                arguments={"message": f"Sync call {i}"}
            )
        sync_time = time.time() - start_time
        
        print(f"✅ Synchronous calls: {num_calls} calls in {sync_time:.2f}s ({sync_time/num_calls:.3f}s per call)")

# Async benchmark
async def benchmark_async_calls(client: MCPClient, num_calls: int = 5):
    """Benchmark async MCP tool calls"""
    
    with client:
        start_time = time.time()
        
        # Concurrent async calls
        tasks = [
            client.call_tool_async(
                tool_use_id=f"bench-async-{i}",
                name="echo",
                arguments={"message": f"Async call {i}"}
            )
            for i in range(num_calls)
        ]
        
        results = await asyncio.gather(*tasks)
        async_time = time.time() - start_time
        
        print(f"✅ Asynchronous calls: {num_calls} concurrent calls in {async_time:.2f}s ({async_time/num_calls:.3f}s per call)")
        print(f"   Speedup: {sync_time/async_time:.1f}x faster")

# Run benchmarks
benchmark_mcp_calls(echo_client)
await benchmark_async_calls(echo_client)

## 15. What's New in MCP Integration

### Recent Enhancements (August-September 2025):

#### ✅ New Features:
- **Async Tool Execution**: Full async support for `call_tool_async()`
- **Prompt Management**: `list_prompts_sync()` and `get_prompt_sync()` methods
- **Pagination Support**: Handle large tool and prompt sets efficiently
- **Structured Content**: Retain structured data from MCP tools
- **Timeout Configuration**: Granular control over startup and execution timeouts
- **OpenTelemetry Instrumentation**: Built-in distributed tracing support

#### 🔧 Improvements:
- Enhanced error handling and resilience
- Better connection management and reuse
- Improved logging and debugging capabilities
- Performance optimizations for concurrent operations

#### 🚀 Coming Soon:
- Resource management support
- Enhanced security features
- More transport options
- Advanced caching mechanisms

In [ ]:
# Summary of new features demonstration
print("🎉 MCP Integration Features Summary:")
print("\n✅ Core Capabilities:")
print("   - Background thread management")
print("   - Multiple transport protocols (stdio, SSE, HTTP)")
print("   - Context manager pattern for resource cleanup")
print("   - Tool discovery and execution")

print("\n🆕 New Features (2025):")
print("   - Async tool execution with call_tool_async()")
print("   - Prompt management (list_prompts_sync, get_prompt_sync)")
print("   - Configurable timeouts (startup_timeout, read_timeout_seconds)")
print("   - Structured content support in tool responses")
print("   - Pagination for large tool/prompt lists")
print("   - OpenTelemetry distributed tracing")

print("\n🔧 Enhanced Capabilities:")
print("   - Client reuse across multiple sessions")
print("   - Robust error handling and retry logic")
print("   - Performance optimizations")
print("   - Comprehensive logging and debugging")

print("\n🌟 Integration Benefits:")
print("   - Access to thousands of pre-built MCP tools")
print("   - Seamless agent-tool communication")
print("   - Enterprise-ready scaling and reliability")
print("   - Cross-platform compatibility")

print("\n✅ MCP Features Guide completed successfully!")

## Conclusion

This guide has covered the comprehensive MCP integration capabilities in the Strands SDK. Key takeaways:

### 🎯 Key Benefits:
- **Extensibility**: Connect to thousands of existing MCP tools
- **Flexibility**: Multiple transport options for different use cases
- **Performance**: Async support and connection reuse
- **Reliability**: Robust error handling and timeout management
- **Observability**: Built-in tracing and logging

### 🚀 Next Steps:
1. Explore the [MCP Server registry](https://github.com/modelcontextprotocol/servers) for available tools
2. Build custom MCP servers for your specific needs
3. Integrate MCP tools into your Strands agents
4. Leverage the new async capabilities for high-performance applications
5. Use structured content for rich data exchange

### 📚 Additional Resources:
- [MCP Protocol Documentation](https://modelcontextprotocol.io/)
- [Strands Documentation](https://strandsagents.com/)
- [MCP Server Examples](https://github.com/modelcontextprotocol/servers)

Happy building with MCP and Strands! 🎉